In [ ]:
from neural_map import NeuralMap, tiles
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('datasets/iris.csv', header=None)
df.columns = ['A', 'B', 'C', 'D', 'Type']
l_columns = ['Type']
col = 'Type'

In [ ]:
scaler = MinMaxScaler()
data = scaler.fit_transform(df.drop(l_columns, axis=1).values)

data.shape

In [ ]:
from numpy import array
from neural_map import mexican_hat
mexican_hat(array([[[0., 0.], [0., 1.]], [[1., 0.], [1., 1.]]]), array([1., 1.]), 1.4, 0.5)

In [ ]:
def no_decay(*args, **kwargs):
    if len(args):
        return args[0]
    else:
        return kwargs['init']

no_decay(init=1)

In [ ]:
som = NeuralMap(variables=data.shape[1], columns=6, rows=6, metric='euclidean', hexagonal=False)

In [ ]:
som.train(data=data, radius_decay_function='exponential', verbose=True, eval_data=data, n_epochs=30)

In [ ]:
som.plot_labels(data, labels=df[col].values, min_cluster_size=4)

In [ ]:
som.plot_unified_distance_matrix(detailed=True, borders=True)

In [ ]:
actfreq, q, mean_distance = som.analyse(data)
umatrix, distance_matrix = som.get_unified_distance_matrix()
labels, probabilities, outlier_socer = som.hdbscan(min_cluster_size=3)

tiles(som.positions, som.hexagonal, umatrix[..., -1], title='distancia')
tiles(som.positions, som.hexagonal, actfreq, title='frecuencia de actvación')
tiles(som.positions, som.hexagonal, q, title='error de cuantización')
tiles(som.positions, som.hexagonal, mean_distance, title='distancia media')
tiles(som.positions, som.hexagonal, probabilities, title = 'probabilidad de prtenencia')
tiles(som.positions, som.hexagonal, outlier_socer, title = 'outiler score')

In [ ]:
clusters = 3

labels, centers = som.k_means(clusters)
tiles(som.positions, som.hexagonal, labels, norm=False, title='Clustering con K-means. Valor de K = ' + str(clusters), labels=list(range(clusters)))

labels, centers = som.k_medoids(clusters)
tiles(som.positions, som.hexagonal, labels, norm=False, title='Clustering con K-medoids. Valor de K = ' + str(clusters), labels=list(range(clusters)))


In [ ]:
som.plot_cluster_weights_vectors(cluster=0)
som.plot_weights_vector(node_index=(0, 0), )

In [ ]:
from BootstrapCCpy import BootstrapCCpy
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

K_CLUSTERS = 5
B_SAMPLINGS = som.columns * som.rows

bcc = BootstrapCCpy(cluster=KMeans().__class__, K=K_CLUSTERS, B=B_SAMPLINGS)
bcc.fit(som.weights.reshape((-1, som.variables)), verbose=True)
bcc.plot_consensus_heatmap()

for clusters in range(2, K_CLUSTERS + 1):
    labels = (fcluster(linkage(bcc.Mk[clusters - 2], 'single'), clusters, criterion='maxclust') - 1).reshape((som.columns, som.rows))
    _plot.tiles(som.positions, som.hexagonal, labels, norm=False, title='BootstrapCC with K = '+ str(clusters), labels=list(range(labels.max() + 1)), intensity=(labels >= 0.) * 1., color_map=plt.cm.get_cmap('hsv', labels.max()), size=4)
    plt.show()

bcc.get_best_k()